In [3]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from model_clip_bert import MultimodalDataset, MultimodalClassifier, evaluate_model
from tqdm import tqdm


C:\Users\dragi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 2. Vérification du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [9]:
# 3. Charger les données
train_df = pd.read_csv("../preprocessing_for_CLIP/train.csv")  # Doit contenir : file_name, text, label
test_texts = pd.read_csv("../preprocessing_for_CLIP/test_texts.csv")
test_labels = pd.read_csv("../preprocessing_for_CLIP/test_labels.csv")
test_df = pd.merge(test_texts, test_labels, on="file_name")

# 4. Séparation train/val
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42)

# 5. Créer les datasets
train_dataset = MultimodalDataset(train_data, image_dir="../Dataset/TRAINING")
val_dataset = MultimodalDataset(val_data, image_dir="../Dataset/TRAINING")
test_dataset = MultimodalDataset(test_df, image_dir="../Dataset/test")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)


In [10]:
# 6. Charger le modèle
model = MultimodalClassifier().to(device)

# 7. Définir l'optimizer et la loss
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

In [11]:
# 8. Entraînement
for epoch in range(3):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        logits = model(
            text_input_ids=batch["text_input_ids"].to(device),
            text_attention_mask=batch["text_attention_mask"].to(device),
            image_pixel_values=batch["image_pixel_values"].to(device),
        )
        loss = criterion(logits, batch["label"].to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")

    val_acc, val_f1 = evaluate_model(model, val_loader, device)
    print(f"Validation Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}")

# 9. Test final
test_acc, test_f1 = evaluate_model(model, test_loader, device)
print("=== Test Results ===")
print(f"Accuracy: {test_acc:.4f}")
print(f"F1 Score : {test_f1:.4f}")

100%|██████████| 1125/1125 [1:32:55<00:00,  4.96s/it]


Epoch 1 - Loss: 0.5261
Validation Accuracy: 0.8160, F1 Score: 0.8333


100%|██████████| 1125/1125 [1:36:04<00:00,  5.12s/it]


Epoch 2 - Loss: 0.3613
Validation Accuracy: 0.8190, F1 Score: 0.8227


100%|██████████| 1125/1125 [1:38:08<00:00,  5.23s/it]


Epoch 3 - Loss: 0.2226
Validation Accuracy: 0.8190, F1 Score: 0.8241
=== Test Results ===
Accuracy: 0.6640
F1 Score : 0.7133
